# Electric vehicle charging station locations - Worldwide 
This notebook has been used to extract API data from Open Charge Map.

API used to extract all recorded charging stations in the world

https://openchargemap.org/site

API documentation: 
https://openchargemap.org/site/develop/api

## Country Codes
Country codes have been retreived from :
https://datahub.io/core/country-list/r/data.json

In [1]:
# Python Dependencies
import requests
import json
import pprint
import pymongo
import time
from config import api_key # Open Charge Map API

ModuleNotFoundError: No module named 'config'

In [2]:
# Get a small record set from the API to better understand the JSON structure 

units = "metric"
country_code = "CN"
count = 1
distanceunit = "KM"
url = f"https://api.openchargemap.io/v3/poi/?key={api_key}&output=json&countrycode={country_code}&distanceunit={distanceunit}&maxresults={count}&compact=true&verbose=false"
# url = "https://api.openchargemap.io/v3/referencedata/"

response = requests.get(url).json()

pprint.pprint(response)

[{'AddressInfo': {'AddressLine1': '7388 Humin Road',
                  'CountryID': 50,
                  'DistanceUnit': 0,
                  'ID': 164114,
                  'Latitude': 31.13140735,
                  'Longitude': 121.40009915,
                  'Title': 'Shanghai - Southern Bailian Supercharger',
                  'Town': 'Shanghai'},
  'Connections': [{'ConnectionTypeID': 27,
                   'CurrentTypeID': 30,
                   'ID': 235354,
                   'LevelID': 3,
                   'PowerKW': 120.0,
                   'Quantity': 10,
                   'StatusTypeID': 50}],
  'DataProviderID': 1,
  'DataQualityLevel': 1,
  'DateCreated': '2020-08-24T16:26:00Z',
  'DateLastStatusUpdate': '2020-08-24T16:27:00Z',
  'GeneralComments': 'Charging10 Superchargers, available 24/7, up to 120kW',
  'ID': 163757,
  'IsRecentlyVerified': False,
  'NumberOfPoints': 10,
  'OperatorID': 23,
  'OperatorsReference': '49613',
  'StatusTypeID': 50,
  'SubmissionStatusT

In [3]:
for location in response:
    print(f"{location['AddressInfo']['Town']} {location['AddressInfo']['Title']} {location['AddressInfo']['Latitude']} {location['AddressInfo']['Longitude']} {location['NumberOfPoints']} {location['GeneralComments']}")
    connections = location['Connections']
    for connection in connections:
        print(f"{connection['CurrentTypeID']} {connection['PowerKW']} {connection['LevelID']} {connection['Quantity']}")

Shanghai Shanghai - Southern Bailian Supercharger 31.13140735 121.40009915 10 Charging10 Superchargers, available 24/7, up to 120kW
30 120.0 3 10


## Country codes
Obtained from datahub.io and stroed in Mongo

In [4]:
# Obtain a list of Country Codes
url = "https://datahub.io/core/country-list/r/data.json"

countries = requests.get(url).json()

In [5]:
pprint.pprint(countries)

[{'Code': 'AF', 'Name': 'Afghanistan'},
 {'Code': 'AX', 'Name': 'Åland Islands'},
 {'Code': 'AL', 'Name': 'Albania'},
 {'Code': 'DZ', 'Name': 'Algeria'},
 {'Code': 'AS', 'Name': 'American Samoa'},
 {'Code': 'AD', 'Name': 'Andorra'},
 {'Code': 'AO', 'Name': 'Angola'},
 {'Code': 'AI', 'Name': 'Anguilla'},
 {'Code': 'AQ', 'Name': 'Antarctica'},
 {'Code': 'AG', 'Name': 'Antigua and Barbuda'},
 {'Code': 'AR', 'Name': 'Argentina'},
 {'Code': 'AM', 'Name': 'Armenia'},
 {'Code': 'AW', 'Name': 'Aruba'},
 {'Code': 'AU', 'Name': 'Australia'},
 {'Code': 'AT', 'Name': 'Austria'},
 {'Code': 'AZ', 'Name': 'Azerbaijan'},
 {'Code': 'BS', 'Name': 'Bahamas'},
 {'Code': 'BH', 'Name': 'Bahrain'},
 {'Code': 'BD', 'Name': 'Bangladesh'},
 {'Code': 'BB', 'Name': 'Barbados'},
 {'Code': 'BY', 'Name': 'Belarus'},
 {'Code': 'BE', 'Name': 'Belgium'},
 {'Code': 'BZ', 'Name': 'Belize'},
 {'Code': 'BJ', 'Name': 'Benin'},
 {'Code': 'BM', 'Name': 'Bermuda'},
 {'Code': 'BT', 'Name': 'Bhutan'},
 {'Code': 'BO', 'Name': 'Bo

In [6]:
# Connect to Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Use the 'electric_vehicles' database in Mongo
db = client.electric_vehicles

# Store in Mongo
db.country_codes.insert_many(countries)

## Charging Station Locations - Per Country
Get the data for each country and store in the Mongo Database

In [7]:
# For testing only - loop through two countries only
countries = [{'Code': 'AU', 'Name': 'Australia'},
 {'Code': 'AT', 'Name': 'Austria'}]
for country in countries:  
    pprint.pprint(country)


{'Code': 'AU', 'Name': 'Australia'}
{'Code': 'AT', 'Name': 'Austria'}


In [8]:
# Connect to Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Use the 'electric_vehicles' database in Mongo
db = client.electric_vehicles

# Set up the Open Charge Map API parameters
units = "metric"
distanceunit = "KM"

# Loop through each country and use the country code for API call
for country in countries: 
    pprint.pprint(f"Trying {country['Name']} ....")
    url = f"https://api.openchargemap.io/v3/poi/?key={api_key}&output=json&countrycode={country['Code']}&distanceunit={distanceunit}"
    country_response = requests.get(url).json()
    
    for location in country_response:
        connections = []
        location_connections = location['Connections']
        for connection in location_connections:
            connection_dict = {'currentTypeID':connection['CurrentTypeID'], 'powerKW':connection['PowerKW'], 'levelID':connection['LevelID'], 'quantity': connection['Quantity']}
            connections.append(connection_dict)
    
        # Prepare the data 
        new_dict = {'country_code':country['Code'], 'town':location['AddressInfo']['Town'], 'location_name':location['AddressInfo']['Title'], 'Latitude':location['AddressInfo']['Latitude'], 'Longitude':location['AddressInfo']['Longitude'], 'charging_points': location['NumberOfPoints'], 'user_comments': location['GeneralComments'], 'connections': connections} 
    
        # Insert a new record in the Charging Station Location collection
        db.charging_station.insert_one(new_dict)
    
    time.sleep(1) # Break between API calls

print("COMPLETE - ALL API data retrieved and stored in Mongo")

'Trying Australia ....'
'Trying Austria ....'
COMPLETE - ALL API data retrieved and stored in Mongo
